In [1]:
import bmdrc
import pandas as pd

## Module 1: Input Data Modules 

##### LPR Class

In [2]:
from bmdrc.LPRClass import LPRClass

lpr_example = pd.read_csv("/Users/degn400/Git_Repos/bmdrc/data/LPR_Long.csv")

LPR = LPRClass(
    df = lpr_example,
    chemical = "chemical.id",
    plate = "plate.id",
    well = "well",
    concentration = "conc",
    time = "variable",
    value = "value",
    cycle_length = 20.0,
    cycle_cooldown = 10.0, 
    starting_cycle = "light"
)

...defining cycles


In [3]:
print(LPR._df)

       chemical.id   conc  plate.id well  variable  value      cycle
0                2    0.0     20544  H01       0.0  0.000     light1
1                2    0.0     20544  H02       0.0    NaN     light1
2                2    0.0     20544  H03       0.0    NaN     light1
3                2    0.0     20544  H04       0.0  1.083     light1
4                2    0.0     20544  H05       0.0  1.608     light1
...            ...    ...       ...  ...       ...    ...        ...
69115            2  100.0     20625  A08     239.0  0.000  gap_dark4
69116            2  100.0     20625  A09     239.0  0.000  gap_dark4
69117            2  100.0     20625  A10     239.0  0.000  gap_dark4
69118            2  100.0     20625  A11     239.0  8.332  gap_dark4
69119            2  100.0     20625  A12     239.0  1.006  gap_dark4

[69120 rows x 7 columns]


In [4]:
print(LPR._cycles)

     variable      cycle
0         0.0     light1
1         1.0     light1
2         2.0     light1
3         3.0     light1
4         4.0     light1
..        ...        ...
235     235.0  gap_dark4
236     236.0  gap_dark4
237     237.0  gap_dark4
238     238.0  gap_dark4
239     239.0  gap_dark4

[240 rows x 2 columns]


In [25]:
import numpy as np

# Group by chemical, concentration, plate, well and time
LPR_grouped = LPR._df.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well, LPR._time])

# Sum times 
LPR_sum = LPR_grouped.sum().rename(columns = {LPR._time:"Sum"}).reset_index()

# Remove times with NA 
LPR_sum["Remove"] = LPR_grouped.apply(lambda df: any(np.isnan(df[LPR._value]))).reset_index().rename(columns = {0:"Remove"})["Remove"]
LPR_sum = LPR_sum[LPR_sum["Remove"] == False].drop(columns = ["Remove"])

In [34]:
# Unique and arrange times 
cycle_info = pd.DataFrame(LPR._df[LPR._time].unique()).rename({0:LPR._time}, axis = 1)
cycle_info[LPR._time] = cycle_info[LPR._time].astype(float)
cycle_info = cycle_info.sort_values(by = [LPR._time])

# Build cycle names and order. First, define all the needed variables to make this happen
cycle_order = []
first_count = 0
gap_a_count = 0
gap_b_count = 0
second_count = 0
cycle_count = 1
if LPR._starting_cycle == "light":
    other_cycle = "dark"
else:
    other_cycle = "light"

# Cycle through the light, gap, dark, and then reset
for pos in range(len(cycle_info)):
    if (first_count < LPR._cycle_length):
        cycle_order.append(LPR._starting_cycle + str(cycle_count))
        first_count += 1
    elif (gap_a_count < LPR._cycle_cooldown):
        cycle_order.append("gap_" + LPR._starting_cycle + str(cycle_count))
        gap_a_count += 1
    elif (second_count < LPR._cycle_length):
        cycle_order.append(other_cycle + str(cycle_count))
        second_count += 1
    elif (gap_b_count < LPR._cycle_cooldown):
        cycle_order.append("gap_" + other_cycle + str(cycle_count))
        gap_b_count += 1
    else:
        cycle_count += 1
        cycle_order.append(LPR._starting_cycle + str(cycle_count))
        first_count = 1
        gap_a_count = 0
        second_count = 0
        gap_b_count = 0
    
# Add essential order information to cycle_info file
cycle_info["cycle"] = cycle_order

# Merge with data.frame
LPR._df = LPR._df.merge(cycle_info)



In [22]:
# Group by chemical, concentration, plate, and well. 
LPR_sum_grouped = LPR_sum.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well])
